# Talent Scheduling

In a talent scheduling problem, we are given a set of scenes $S = \{ 0, ..., n - 1 \}$ and a set of actors $A = \{ 0, ..., m - 1 \}$.
In a scene $s \in S$, a set of actors $A_s \subseteq A$ plays for $d_s$ days.
An actor comes to the location when the first scene he or she plays starts and leaves when the last scene he or she plays ends.
For each day actor $a$ is on location, we need to pay the cost $c_a$.
We want to find a sequence of scenes to shoot such that the total cost is minimized.

## DP Formulation

Suppose that a set of scenes $Q$ is remaining.
A set of actors $\bigcup_{s \in N \setminus Q} A_s$ already came to the location, and $\bigcup_{s \in Q} A_s$ is still on location because they need to play on the remaining scenes $Q$.
Therefore, if we shoot a scene $s \in Q$ next, the set of actors on location will be

$$
    L(s, Q) = A_s \cup \left( \bigcup_{s' \in N \setminus Q} A_{s'} \cap \bigcup_{s' \in Q } A_{s'}  \right).
$$

We need to pay the cost $d_s \sum_{a \in L(s, Q)} c_a$ when shooting scene $s$.
Once we shot scene $s$, the remaining problem is to decide the order of the remaining scenes $Q \setminus \{ s \}$.
Therefore, a state is defined by the set of remaining scenes $Q$, and the minimum cost to shoot $Q$ is represented by $V(Q)$.
Because $A_s$, actors who play in scence $s$, are always on location when $s$ is shot, $\sum_{s \in Q} d_s \sum_{a \in A_s} c_a$ is a lower bound on $V(Q)$.
We have the following DP formulation.

$$
\begin{align}
    \text{compute } & V(N) \\
    & V(Q) = \begin{cases}
        \min\limits_{s \in Q} d_s \sum\limits_{a \in L(s, Q)} c_a + V(Q \setminus \{ s \}) & \text{if } Q \neq \emptyset \\
        0 & \text{if } Q = \emptyset
    \end{cases} \\
    & V(Q) \geq \sum_{s \in Q} d_s \sum_{a \in A_s} c_a.
\end{align}
$$

If $A_s$, the set of actors that play in scence $s$, is equivalent to the set of actors currently on location, we can shoot $s$ with the minimum cost:
we just need to pay for the actors who play in $s$.
We should always shoot such a scene first.
In state $Q$, the set of actors on location is $\bigcup_{s \in N \setminus Q} A_{s} \cap \bigcup_{s \in Q} A_{s}$.
Therefore, we have the following equation in addition:

$$
    V(Q) = d_s \sum\limits_{a \in A_s} c_a + V(Q \setminus \{ s \}) \text{ if } s \in Q \land A_s = \bigcup_{s' \in N \setminus Q} A_{s'} \cap \bigcup_{s' \in Q} A_{s'}.
$$

## Data

In [1]:
# Number of scenes
n = 4
# Number of actors
m = 4
# Duration of scenes
d = [1, 1, 1, 1]
# Costs of actors
c = [1, 3, 1, 2]
# Actors in each scene
scene_to_actors = [[0, 1, 3], [1, 2], [0, 2, 3], [0, 1, 2]]

## Modeling

In [2]:
# !pip install didppy

In [3]:
import didppy as dp


model = dp.Model()

scene = model.add_object_type(number=n)
actor = model.add_object_type(number=m)

# Q
remaining = model.add_set_var(object_type=scene, target=list(range(n)))

scene_to_actors_table = model.add_set_table(scene_to_actors, object_type=scene)
actor_to_cost = model.add_int_table(c)

# Precompute the minimum cost of each scene
scene_to_min_cost = model.add_int_table(
    [d[s] * sum(c[a] for a in scene_to_actors[s]) for s in range(n)]
)

for s in range(n):
    already_shot = remaining.complement()
    came_to_location = scene_to_actors_table.union(already_shot)
    standby = scene_to_actors_table.union(remaining)
    on_location = scene_to_actors_table[s] | (came_to_location & standby)

    shoot = dp.Transition(
        name="shoot {}".format(s),
        cost=d[s] * actor_to_cost[on_location] + dp.IntExpr.state_cost(),
        preconditions=[remaining.contains(s)],
        effects=[(remaining, remaining.remove(s))],
    )
    model.add_transition(shoot)

model.add_base_case([remaining.is_empty()])

model.add_dual_bound(scene_to_min_cost[remaining])

for s in range(n):
    already_shot = remaining.complement()
    came_to_location = scene_to_actors_table.union(already_shot)
    standby = scene_to_actors_table.union(remaining)
    on_location = scene_to_actors_table[s] | (came_to_location & standby)

    shoot = dp.Transition(
        name="forced shoot {}".format(s),
        cost=scene_to_min_cost[s] + dp.IntExpr.state_cost(),
        preconditions=[
            remaining.contains(s),
            scene_to_actors_table[s] == (came_to_location & standby),
        ],
        effects=[(remaining, remaining.remove(s))],
    )
    model.add_transition(shoot, forced=True)

## Solving

In [4]:
solver = dp.CABS(model, quiet=True)
solution = solver.search()

print("Transitions to apply:")
print("")

for t in solution.transitions:
    print(t.name)

print("")
print("Cost: {}".format(solution.cost))

Transitions to apply:

shoot 2
shoot 0
forced shoot 3
forced shoot 1

Cost: 20
